In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.debugger import Tracer # debugging
from IPython.display import clear_output
import ipywidgets as widgets
import time

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

import numpy as np
import sklearn.gaussian_process as gp

In [ ]:
# local modules
import sys; sys.path.append('..')
import optimisation as op

In [ ]:
# make deterministic
np.random.seed(100)

Function to optimize:

In [ ]:
f = lambda x: 1 * x * np.cos(x)
xs = np.linspace(0, 12, num=200)

ys = f(xs)
best_y = np.min(ys)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(xs, ys, 'g-')
plt.margins(0.1, 0.1)
plt.xlabel('x')
plt.ylabel('cost')
plt.show()

In [ ]:
ranges = {
    'x': xs,
}
Surrogate = op.SciKitGPSurrogate.Custom(gp_params=dict(
    alpha = 1e-3, # larger => more noise. Default = 1e-10
    kernel = 1.0 * gp.kernels.RBF(),
    n_restarts_optimizer = 10,
    normalize_y = True,
    copy_X_train = True # make a copy of the training data
))
strategy = op.AcquisitionStrategy(
    pre_phase_steps = 3,
    acquisition_function = ('UCB', {'kappa' : 3})
)

In [ ]:
class MyEvaluator(op.Evaluator):
    def test_config(self, config):
        return f(config.x)

In [ ]:
optimiser = op.BayesianOptimisationOptimiser(ranges, maximise_cost=False, acquisition_strategy=strategy,
                                             Surrogate=Surrogate, close_tolerance=1e-3)
evaluator = MyEvaluator()

In [ ]:
op.gui.optimiser_progress_bar(optimiser)
%time optimiser.run_sequential(evaluator, max_jobs=15)

# or run interactively to view the log as it runs
#task = lambda: optimiser.run_sequential(evaluator, max_jobs=20)
#op.interactive(optimiser, task)

In [ ]:
optimiser.plot_cost_over_time(true_best=best_y);

In [ ]:
def plot(n, step):
    optimiser.plot_step_1D('x', n, true_cost=f);
op.gui.step_log_slider(optimiser, plot, pre_compute=False);

# Try optimising the same function with random search

In [ ]:
ra = op.RandomSearchOptimiser(ranges, maximise_cost=False)
ra.run_sequential(evaluator, max_jobs=15)

In [ ]:
ra.plot_cost_over_time(true_best=best_y).set_size_inches(16, 6)

# Plot the samples to compare with the Bayesian samples

In [ ]:
r_xs = [s.config.x for s in ra.samples]
r_ys = [s.cost for s in ra.samples]

plt.figure(figsize=(16,6))
plt.plot(xs, ys, 'g-')
plt.plot(r_xs, r_ys, 'bo', zorder=10)
plt.margins(0.1, 0.1)
plt.xlabel('x')
plt.ylabel('cost')
plt.show()